# Generate patches

In [1]:
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
from patch_extraction import extract_patches
import numpy as np
import sqlite3
import pandas as pd
from glob import glob
import os

from sqlalchemy import create_engine

In [2]:
folder = "C:/Users/caspe/Desktop/patches/"
images = glob(folder + '*.tif')
for image in images:
    name = os.path.splitext(os.path.basename(image))[0]
    extract_patches(
        image,
        folder + f"{name}.npy",
        size=16,
        overlaps=[(8, 0), (8, 8), (0, 8)],
        fill_value=0,
    )


Generating blocks..
Writing numpy array to disc..


In [3]:
# Load data
in_path = folder + "ground_truth.sqlite"

db_cnx = sqlite3.connect(in_path)
df = pd.read_sql_query("SELECT * FROM 'ground_truth';", db_cnx)
df

,ogc_fid,fid,people,area,volume,muni_code
0,1,2551,0.896936,43.584155,93.980530,779.0
1,2,2582,0.000000,0.000000,0.000000,779.0
2,3,2583,0.000000,0.000000,0.000000,779.0
3,4,2588,0.000000,0.000000,0.000000,779.0
4,5,2589,0.000000,0.000000,0.000000,779.0
...,...,...,...,...,...,...
1974292,1974293,3483934,0.000000,0.000000,0.000000,766.0
1974293,1974294,3483935,0.000000,0.000000,0.000000,766.0
1974294,1974295,3483936,1.983452,1872.618227,11251.704869,766.0
1974295,1974296,3483937,0.000000,275.866573,1650.423805,766.0


In [4]:
df['id'] = df.index

In [5]:
y = df[['id', 'fid', 'muni_code', 'volume', 'area', 'people']]
# np.save(folder + "ground_truth.npy", y.values)

In [6]:
images = glob(folder + '*.npy')
for image in images:
    name = os.path.splitext(os.path.basename(image))[0]
    x = np.load(image)
    np.save(folder + f"{name}_project_area.npy", x[y['fid'].values])

In [7]:
bv = np.load(folder + "structural_volume_mult25_project_area.npy")
# b8 = np.load(folder + "b08_project_area.npy")

In [8]:
bv.shape

(1974297, 16, 16)

In [ ]:
all_images = np.stack([
    np.load(folder + "b04_project_area.npy"),
    np.load(folder + "b04_tex_project_area.npy"),
    np.load(folder + "b08_project_area.npy"),
    np.load(folder + "b08_tex_project_area.npy"),
    np.load(folder + "bs_asc_project_area.npy"),
    np.load(folder + "bs_desc_project_area.npy"),
    np.load(folder + "coh_asc_project_area.npy"),
    np.load(folder + "coh_desc_project_area.npy"),
    np.load(folder + "nl_project_area.npy"),
], axis=1)
all_images.shape

In [ ]:
np.save(folder + "all_project_area.npy", all_images)